# Implementation of the SIMCLR with resnet50 backbone

In [1]:
import wandb
wandb.login()  # Opens a browser once to authenticate


wandb: Currently logged in as: analiju (analiju-paris) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
import os
import ssl
import zipfile
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, random_split, Dataset, Subset
from torchvision.models import resnet50
from PIL import Image
import numpy as np
import random
from sklearn.model_selection import StratifiedShuffleSplit
from torch.optim.lr_scheduler import LinearLR, CosineAnnealingLR, SequentialLR
import seaborn as sns


In [ ]:

# Prevent nondeterminism
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
torch.backends.cudnn.enabled = False

CONFIG = {
    "LOCAL_OR_COLAB": "LOCAL",
    "DATA_DIR_LOCAL": "/share/DEEPLEARNING/carvalhj/EuroSAT_RGB/",
    "DATA_DIR_COLAB": "/content/EuroSAT_RGB",
    "ZIP_PATH": "/content/EuroSAT.zip",
    "EUROSAT_URL": "https://madm.dfki.de/files/sentinel/EuroSAT.zip",
    "SEED": 42,  
    "BATCH_SIZE": 128,
    "LR": 3.75e-4,
    "WD": 0.5,
    "LR_LINEAR": 3.75e-4,
    "EPOCHS_SIMCLR": 10,
    "EPOCHS_LINEAR": 10,
    "PROJ_DIM": 128,
    "FEATURE_DIM": 2048, # ResNet50 feature dimension = 2048
}

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# split fractions
TRAIN_FRAC = 0.8
VAL_FRAC   = 0.1
TEST_FRAC  = 0.1

SEED = CONFIG["SEED"]

PRETRAINED = False

TEMPERATURE = 0.2

BETAS=(0.9,0.98)
EPS = 1e-8

LINEAR_PROB_TRAIN_SPLIT = 0.75

GLOBAL_SEED = CONFIG["SEED"]
NUM_WORKERS = 4

In [ ]:

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark     = False

def prepare_data():
    if CONFIG["LOCAL_OR_COLAB"] == "LOCAL":
        return CONFIG["DATA_DIR_LOCAL"]

    if not os.path.exists(CONFIG["DATA_DIR_COLAB"]):
        print("Downloading EuroSAT RGB...")
        ssl._create_default_https_context = ssl._create_unverified_context
        urllib.request.urlretrieve(CONFIG["EUROSAT_URL"], CONFIG["ZIP_PATH"])
        with zipfile.ZipFile(CONFIG["ZIP_PATH"], 'r') as zip_ref:
            zip_ref.extractall("/content")
        os.rename("/content/2750", CONFIG["DATA_DIR_COLAB"])
        print("EuroSAT RGB dataset downloaded and extracted.")
    return CONFIG["DATA_DIR_COLAB"]


def compute_mean_std(dataset, batch_size):
    loader = DataLoader(dataset, batch_size, shuffle=False, num_workers=2)
    mean = 0.0
    std = 0.0
    n_samples = 0

    for data, _ in loader:
        batch_samples = data.size(0)
        data = data.view(batch_samples, data.size(1), -1)  # (B, C, H*W)
        mean += data.mean(2).sum(0)
        std += data.std(2).sum(0)
        n_samples += batch_samples

    mean /= n_samples
    std /= n_samples
    return mean.tolist(), std.tolist()


class TwoCropsTransform:
    def __init__(self, base_transform):
        self.base_transform = base_transform

    def __call__(self, x):
        return [self.base_transform(x), self.base_transform(x)]
    
class SimCLRDataset(Dataset):
    def __init__(self, dataset, transform):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        x, _ = self.dataset[idx]
        x1, x2 = self.transform(x)
        return x1, x2

def get_split_indexes(labels, total_count):
    indices = np.arange(total_count)
    np.random.seed(SEED) # for reproducibility
    np.random.shuffle(indices)

    train_split = int(0.8 * total_count)
    val_split = int(0.9 * total_count)

    train_idx = indices[:train_split]
    val_idx = indices[train_split:val_split]
    test_idx = indices[val_split:]
    return train_idx, val_idx, test_idx


def get_data_loaders(data_dir, batch_size):

    dataset_for_stats = datasets.ImageFolder(
        root=data_dir,
        transform=transforms.ToTensor()
    )
    total_len = len(dataset_for_stats)
    n_train = int(TRAIN_FRAC * total_len)
    n_val   = int(VAL_FRAC   * total_len)
    n_test  = total_len - n_train - n_val

    train_for_stats, val_for_stats, test_for_stats = random_split(
        dataset_for_stats,
        [n_train, n_val, n_test]
    )
    
    mean, std = compute_mean_std(train_for_stats, batch_size)

    dataset_train_no_transform = datasets.ImageFolder(
        root=data_dir,
        transform=None
    )

    generator = torch.Generator().manual_seed(seed)
    train_indices, val_indices, test_indices = random_split(
        list(range(total_len)),
        [n_train, n_val, n_test],
        generator=generator
    )
    
    all_indices = list(range(total_len))
    rnd = random.Random(seed)
    rnd.shuffle(all_indices)
    train_indices = all_indices[:n_train]
    val_indices   = all_indices[n_train : n_train + n_val]
    test_indices  = all_indices[n_train + n_val : ]

    # Now build Subsets pointing to dataset_train_no_transform:
    train_subset_no_transform = Subset(dataset_train_no_transform, train_indices)

    # 3) IMAGEFOLDER FOR VALIDATION/TEST (WITH EVAL TRANSFORM)
    eval_transform = transforms.Compose([
        transforms.Resize(72),
        transforms.CenterCrop(64),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ])
    dataset_eval = datasets.ImageFolder(
        root=data_dir,
        transform=eval_transform
    )
    val_subset = Subset(dataset_eval, val_indices)
    test_subset = Subset(dataset_eval, test_indices)

    # DEFINE SIMCLR TRANSFORMS FOR TRAIN
    normalize = transforms.Normalize(mean=mean, std=std)
    augment_transform = transforms.Compose([
        transforms.RandomResizedCrop(64, scale=(0.5, 1.0)),
        transforms.RandomHorizontalFlip(),
    
        transforms.RandomGrayscale(p=0.2),
        transforms.ToTensor(),
        normalize,
    ])
    simclr_transform = TwoCropsTransform(augment_transform)
    train_ds_simclr = SimCLRDataset(train_subset_no_transform, simclr_transform)

    train_loader = DataLoader(
        train_ds_simclr,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=2
    )
    val_loader = DataLoader(
        val_subset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2
    )
    test_loader = DataLoader(
        test_subset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2
    )

    num_classes = len(dataset_for_stats.classes)
    return train_loader, val_loader, test_loader, num_classes

class ProjectionHead(nn.Module):
    def __init__(self, input_dim, proj_dim=128, hidden_dim=2048):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, proj_dim)
        )

    def forward(self, x):
        return self.net(x)

class SimCLRModel(nn.Module):
    def __init__(self, base_encoder, proj_dim=128):
        super().__init__()
        self.encoder = base_encoder
        self.encoder.fc = nn.Identity()
        self.projection_head = ProjectionHead(input_dim=CONFIG["FEATURE_DIM"], proj_dim=proj_dim)

    def forward(self, x):
        feat = self.encoder(x)
        proj = self.projection_head(feat)
        return feat, proj

class NTXentLoss(nn.Module):
    def __init__(self, batch_size, temperature=0.5, device='cuda'):
        super().__init__()
        self.temperature = temperature
        self.batch_size = batch_size
        self.device = device
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, zis, zjs):
        N = zis.size(0)
        z = F.normalize(torch.cat([zis, zjs], dim=0), dim=1)
        sim = torch.matmul(z, z.T) / self.temperature
        mask = torch.eye(2 * N, dtype=torch.bool).to(self.device)
        sim = sim.masked_fill(mask, -1e9)
        labels = torch.cat([torch.arange(N, 2 * N), torch.arange(0, N)]).to(self.device)
        # return self.x(sim, labels)
        return self.criterion(sim, labels)

def train_simclr(model, train_loader, val_loader, optimizer, criterion, device, epochs, scheduler=None):
    model.train()
    model.to(device)
    # Start a W&B run here
   
    for epoch in range(epochs):
        total_loss = 0
        for (x1, x2) in train_loader:
            x1, x2 = x1.to(device), x2.to(device)
            _, z1 = model(x1)
            _, z2 = model(x2)
            loss = criterion(z1, z2)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if scheduler:
                scheduler.step()
            total_loss += loss.item()
        avg = total_loss / len(train_loader)
        # print(f"[SimCLR] Epoch {epoch+1}/{epochs} - Loss: {avg:.4f}")
        # wandb.log({"epoch": epoch + 1, "loss": avg})
        # Evaluate on validation set
        # val_accuracy = evaluate(model.encoder, model.projection_head, val_loader, device)
        # print(f"[SimCLR] Validation Accuracy: {val_accuracy:.2f}%")
        # wandb.log({"val_accuracy": val_accuracy})
        print(f"[SimCLR] Epoch {epoch+1}/{epochs} - Train Loss: {avg:.4f}%")
        
    print("Finished SimCLR pretraining.")

def evaluate(classifier, backbone, loader, device):
    classifier.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            features = backbone(images)
            outputs = classifier(features)
            total += labels.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()
    return correct / total * 100

def make_optimizer_scheduler(params, lr, wd, steps_per_epoch, epochs):
    opt = optim.Adam(params.parameters(), lr=CONFIG["LR"])
    return opt, None

In [ ]:
def train_linear_probe(backbone, train_loader, val_loader, device, epochs, lr, run_id):
    for p in backbone.parameters():
        p.requires_grad = False

    base_ds = train_loader.dataset
    while isinstance(base_ds, Subset):
        base_ds = base_ds.dataset
    num_classes = len(base_ds.classes)

    classifier = nn.Linear(CONFIG["FEATURE_DIM"], num_classes).to(device)

    optimizer, _ = make_optimizer_scheduler(
        classifier.parameters(),
        lr=lr,
        wd=CONFIG["WD"],
        steps_per_epoch=len(train_loader),
        epochs=epochs
    )
    wandb.init(
        project="EuroSAT_SimCLR_LinearProbe",
        name=f"LinearProbe_Seed{run_id}",
        config=CONFIG,
    )
    wandb.watch(classifier, log="all", log_freq=100)

    criterion = nn.CrossEntropyLoss()


    for epoch in range(epochs):
        classifier.train()
        correct, total = 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            features = backbone(images)
            outputs = classifier(features)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total += labels.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()

        train_acc = correct / total * 100
        val_acc = evaluate(classifier, backbone, val_loader, device)
        print(f"[Linear] Epoch {epoch+1}/{epochs} - "
              f"Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")

    torch.save(classifier.state_dict(), f"linear_probe_seed{run_id}.pth")
    return val_acc

def load_evaluate_model(model_path, device, data_dir, seed):

    wandb.init(
        project="EuroSAT_SimCLR_LinearProbe",
        name=f"LinearProbe_Seed{seed}",
        config=CONFIG,
    )
    results = []

    backbone = resnet50(weights=None if not PRETRAINED else "DEFAULT")
    backbone.fc = nn.Identity()  # same as in SimCLRModel
    backbone.load_state_dict(torch.load(model_path), strict=False)
    backbone.to(device)    
    backbone.eval()
    for p in backbone.parameters():
        p.requires_grad = False

    _, _, test_loader, _ = get_data_loaders(data_dir, CONFIG["BATCH_SIZE"])
    print(f"Starting linear probe on EuroSAT test split (seed={seed})...")

    full_test_ds = test_loader.dataset  # this is a Subset of dataset_eval
    train_size = int(LINEAR_PROB_TRAIN_SPLIT * len(full_test_ds))
    val_size = len(full_test_ds) - train_size
    train_dataset, val_dataset = random_split(full_test_ds, [train_size, val_size])

    train_loader_from_test = DataLoader(
        train_dataset,
        batch_size=CONFIG["BATCH_SIZE"],
        shuffle=True,
        num_workers=2
    )
    val_loader_from_test = DataLoader(
        val_dataset,
        batch_size=CONFIG["BATCH_SIZE"],
        shuffle=False,
        num_workers=2
    )

    linear_probe_val_acc = train_linear_probe(
        backbone,
        train_loader_from_test,
        val_loader_from_test,
        DEVICE,
        epochs=CONFIG["EPOCHS_LINEAR"],
        lr=CONFIG["LR_LINEAR"],
        run_id=seed
    )
    wandb.log({"linear_probe_val_acc": linear_probe_val_acc})
    wandb.finish()
    print(f"[Linear‐Probe on EuroSAT test] Final Val Acc = {linear_probe_val_acc:.2f}%\n")

    results.append({
        "seed": seed,
        "val_acc": linear_probe_val_acc
    })
    with open("linear_probe_results.txt", "a") as f:
        f.write(f"Seed: {seed}, Val Acc: {linear_probe_val_acc:.2f}%\n")
    print("Results saved to linear_probe_results.txt")
    return results

In [6]:

seeds = [GLOBAL_SEED]

for seed in seeds:
    print(f"\n=== Starting run with seed {seed} ===")
    set_seed(seed)
    
    data_dir = prepare_data()
    train_loader, val_loader, test_loader, num_classes = get_data_loaders(data_dir, CONFIG["BATCH_SIZE"])

    # Initialize base encoder and SimCLR model
    base_encoder = resnet50(weights=None)
    simclr_model = SimCLRModel(base_encoder, proj_dim=CONFIG["PROJ_DIM"])
    # optimizer = optim.Adam(simclr_model.parameters(), lr=CONFIG["LR"])
    wd =  0.5 
    optimizer, scheduler = make_optimizer_scheduler(
        simclr_model.parameters(),
        CONFIG["LR"],
        CONFIG["WD"],
        len(train_loader),
        CONFIG["EPOCHS_SIMCLR"]
        )
    
    bs = CONFIG["BATCH_SIZE"]
    loss_fn = NTXentLoss(bs, temperature=TEMPERATURE, device=DEVICE)

    print("Starting SimCLR training...")
    lr = CONFIG["LR"]
    wandb_run = wandb.init(
        project="eurosat-contrastive-scratch-grid-search",
        name=f"BS{bs}_LR{lr:.0e}_SEED{seed}_TEMPERATURE{TEMPERATURE}",
        config={
            "seed": seed,
            "temperature": TEMPERATURE,
            "model": "SimCLR",
            "dataset": "EuroSAT",
            "batch_size": bs,
            "learning_rate": CONFIG["LR"],
            "epochs": CONFIG["EPOCHS_SIMCLR"],
            "proj_dim": CONFIG["PROJ_DIM"],
            "feature_dim": CONFIG["FEATURE_DIM"],
            "pretrained": PRETRAINED,
        }
    )
    train_simclr(simclr_model, train_loader, val_loader, optimizer, loss_fn, DEVICE, CONFIG["EPOCHS_SIMCLR"], scheduler=scheduler)
    wandb_run.finish()

    print("Saving encoder...")
    torch.save(simclr_model.state_dict(), f"simclr_model_seed{seed}_temperature{TEMPERATURE}_bs{bs}.pth")



=== Starting run with seed 42 ===


Starting SimCLR training...


[SimCLR] Epoch 1/10 - Train Loss: 3.5432%


/users/c/carvalhj/miniconda3/envs/myenv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


[SimCLR] Epoch 2/10 - Train Loss: 2.1323%
[SimCLR] Epoch 3/10 - Train Loss: 1.8009%
[SimCLR] Epoch 4/10 - Train Loss: 1.6473%
[SimCLR] Epoch 5/10 - Train Loss: 1.5703%
[SimCLR] Epoch 6/10 - Train Loss: 1.5027%
[SimCLR] Epoch 7/10 - Train Loss: 1.4596%
[SimCLR] Epoch 8/10 - Train Loss: 1.4277%
[SimCLR] Epoch 9/10 - Train Loss: 1.4083%
[SimCLR] Epoch 10/10 - Train Loss: 1.3981%
Finished SimCLR pretraining.


Saving encoder...


In [7]:

# Run the evaluation
for seed in seeds:
    results = load_evaluate_model(f"simclr_model_seed{seed}.pth", DEVICE, data_dir, seed)
    print(f"Results for seed {seed}: {results}")
    

/tmp/ipykernel_18041/781684196.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  backbone.load_state_dict(torch.load(model_path), strict=False)


Starting linear probe on EuroSAT test split (seed=42)...


/users/c/carvalhj/miniconda3/envs/myenv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


[Linear] Epoch 1/10 - Train Acc: 9.93%, Val Acc: 12.30%
[Linear] Epoch 2/10 - Train Acc: 11.16%, Val Acc: 6.81%
[Linear] Epoch 3/10 - Train Acc: 12.49%, Val Acc: 14.37%
[Linear] Epoch 4/10 - Train Acc: 17.83%, Val Acc: 19.70%
[Linear] Epoch 5/10 - Train Acc: 21.33%, Val Acc: 20.00%
[Linear] Epoch 6/10 - Train Acc: 25.48%, Val Acc: 24.59%
[Linear] Epoch 7/10 - Train Acc: 25.78%, Val Acc: 25.04%
[Linear] Epoch 8/10 - Train Acc: 27.46%, Val Acc: 27.85%
[Linear] Epoch 9/10 - Train Acc: 28.25%, Val Acc: 24.74%
[Linear] Epoch 10/10 - Train Acc: 27.75%, Val Acc: 24.74%


linear_probe_val_acc,▁
linear_probe_val_acc,24.74074


[Linear‐Probe on EuroSAT test] Final Val Acc = 24.74%

Results saved to linear_probe_results.txt
Results for seed 42: [{'seed': 42, 'val_acc': 24.74074074074074}]
